<a href="https://colab.research.google.com/github/LaineWishart/Degree/blob/master/Laine_Country_analysis_template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import io
import matplotlib.pyplot as plt
from typing import List, Dict
!pip install sigfig
import sigfig
import scipy
import plotly.graph_objects as go
import plotly.io as pio
from datetime import datetime
from plotly.subplots import make_subplots
import scipy.stats as sp
import random 
import datetime
import plotly.express as px
from google.colab import data_table
import warnings
import sys

# Functions

Visualisation function creation and testing with synthetic dataset can be found [here ](https://colab.research.google.com/drive/1NxqhRKehgFFrbNrYUSpw5FzsLr3rN_8f?usp=sharing).

In [ ]:
# Created by Lane, 2020-11-05, completed on 2020-11-07

def get_monthly_totals(data, country, variable_name):
  """Returns monthly totals for a given variable."""

  # convert country to lowercase
  country = country.lower().replace(" ", "_")
  
  # get the index values
  multi_index_vals =  list(data.index.get_level_values(level=0).unique())

  # check the time variable is in datetime format
  assert np.issubdtype(data.loc[:, 'time'], np.datetime64), '"time" column dtype should be datetime64[ns]'

  # check the country is in the multi-index value list
  assert country in multi_index_vals, "{country} not in the index.".format(country=country)

  # check the input variable is in the columns
  assert variable_name in data.columns, "Input variable not found in dataframe columns. "

  # add a month column to the data
  data['month'] = pd.DatetimeIndex(data['time']).month

  # calculate the monthly totals for the given variable
  monthly_totals_df =  all_countries.loc[country][['month', variable_name]].groupby('month').sum()

  return monthly_totals_df

def get_monthly_stats(data, country, variable_name):

  # convert country to lowercase
  country = country.lower().replace(" ", "_")
  
  # get the index values
  multi_index_vals =  list(data.index.get_level_values(level=0).unique())

   # check the time variable is in datetime format
  assert np.issubdtype(data.loc[:, 'time'], np.datetime64), '"time" column dtype should be datetime64[ns]'

  # check the country is in the multi-index value list
  assert country in multi_index_vals, "{country} not in the index.".format(country=country)

  # check the input variable is in the columns
  assert variable_name in data.columns, "Input variable not found in dataframe columns. "
  
  # get vars from df for selected country
  data_subset = data.loc[country][['time', variable_name]]

  # make time the index
  data_subset.index = data_subset['time']
  data_subset.drop('time', axis=1, inplace=True)

  # calculate the mean, sd, and n for each var
  mean = data_subset.astype('float').resample('M').mean().T
  sd = data_subset.astype('float').resample('M').std().T
  n = data_subset.astype('float').resample('M').count().astype(int).T
  med = data_subset.astype('float').resample('M').median().T
  max = data_subset.astype('float').resample('M').max().T
  min = data_subset.astype('float').resample('M').min().T
  
  # put all the stats together
  monthly_stats_list = pd.concat([n,
                       max,
                       min, 
                       med,
                       mean, 
                       sd ],
                       keys = [
                              'count',
                              'max',
                              'min', 
                              'median', 
                              'mean', 
                              'standard deviation'
                               ]
                      ).reset_index()

  # replace index with multi-level index
  multi_idx = pd.MultiIndex.from_frame(monthly_stats_list.loc[:, ['level_0']], names = ["stat"])
  monthly_stats_list= monthly_stats_list.set_index(multi_idx).drop(['level_0', 'level_1'], axis=1)
  monthly_stats_list.sort_values(['stat'], inplace=True)

  # round to nearest decimal place
  monthly_stats_list = round(monthly_stats_list, 1)

  return monthly_stats_list


def analyse_one_var(data, country, variable_name, nbins = 10):
  # convert country to lowercase
  country = country.lower().replace(" ", "_")
  
  # get the index values
  multi_index_vals =  list(data.index.get_level_values(level=0).unique())

   # check the time variable is in datetime format
  assert np.issubdtype(data.loc[:, 'time'], np.datetime64), '"time" column dtype should be datetime64[ns]'

  # check the country is in the multi-index value list
  assert country in multi_index_vals, "{country} not in the index.".format(country=country)

  # check the input variable is in the columns
  assert variable_name in data.columns, "Input variable not found in dataframe columns. "

  # get the monthly totals for the histogram 
  monthly_totals = get_monthly_totals(data, country, variable_name)

  # get monthly summary stats for the table 
  monthly_stats = get_monthly_stats(data, country, variable_name)

  # assign a colour to each variable
  colour_dict = {'deaths': '#EF553B' , 'cases': '#636EFA', 'tests': '#00CC96', 'mortality_rate': '#FFA15A', 'case_fatality_rate': '#FF97FF'}

  # get the colour for the selected country
  colour = colour_dict[variable_name]

  # assign values to x and y variables
  x = data.loc[country]['time']
  y = data.loc[country][variable_name]

  subplot_titles = ("Daily {var} over time".format(var = variable_name.replace("_", " ")),
                    "Number of months with same monthly {var}".format(var = variable_name.replace("_", " ")),
                    "Monthly summary statistics for COVID-19 {var}".format(var=variable_name.replace("_", " "))) 

  fig = make_subplots(
    rows=2, cols=2,
    specs=[[{"type": "scatter"}, {"type": "histogram"}],
           [{"colspan": 2,  "type": "table"}, None]],
    subplot_titles= subplot_titles, 
    vertical_spacing = 0.15, 
    horizontal_spacing = 0.15)

  # add a table
  fig.add_trace(
      go.Table(header = dict(values = ["",
                                      "December 2019",
                                      "January 2020",
                                      "February 2020",
                                      "March 2020",
                                      "April 2020",
                                      "May<br>2020",
                                      "June 2020", 
                                      "July<br>2020", 
                                      "August 2020", 
                                      "September 2020", 
                                      "October 2020", 
                                      "November 2020"], 
                                line_color='darkslategray',
      fill_color='aliceblue',
      align=['left','center'],
      font=dict(color='black', size=12),
      height=50), 
              cells = dict(
                  values = [[ 'n days with data',
                              'max',
                              'min', 
                              'median', 
                              'mean', 
                              'standard deviation'
                               ]] + [monthly_stats[k].tolist() for k in monthly_stats.columns[0:]], 
                      line_color='darkslategray',
      fill=dict(color=['aliceblue', 'white']),
      align=['left', 'center'],
      font_size=12,
      height=20)),  
      row = 2, col=1
  )

  # add a line plot
  fig.add_trace(go.Scatter(
      x=x,
      y=y,
      name='fig1',
      line=dict(color=colour, width=2)
  ), row = 1, col=1)

  # add histogram
  fig.add_trace(go.Histogram(name=None,
       y = monthly_totals.loc[:, variable_name],
      nbinsy = nbins, 
      ybins=dict(
          start=np.min(monthly_totals.loc[:, variable_name]),
          end=np.max(monthly_totals.loc[:, variable_name])
                 ), 
                 marker = dict(color = colour)
      ), row = 1, col = 2)
  
  # update the figure layout
  fig.update_layout(
      title="COVID-19 {var} in {country}".format(var = variable_name.replace("_", " "), country = country),
      autosize=True,
      width=1500,
      height=800,
      template = "plotly_white",
      showlegend=False
      )

  fig.update_yaxes(title_text="Daily case fatality rate", row=1, col=1)
  fig.update_xaxes(title_text="Number of months", row=1, col=2)

  fig.update_xaxes(title_text = "Time", row=1, col=1)
  fig.update_yaxes(title_text = "Monthly {var}".format(var = variable_name.replace("_", " ")), row=1, col=2)

  fig.show(renderer = 'colab')


def analyse_two_vars(data, country, first_variable, second_variable):
  # convert country to lowercase
  country = country.lower().replace(" ", "_")
  
  # get the index values
  multi_index_vals =  list(data.index.get_level_values(level=0).unique())

  # check the time variable is in datetime format
  assert np.issubdtype(data.loc[:, 'time'], np.datetime64), '"time" column dtype should be datetime64[ns]'

  # check the country is in the multi-index value list
  assert country in multi_index_vals, "{country} not in the index.".format(country=country)

  # check the input variables are in the columns
  assert first_variable in data.columns, "First input variable not found in dataframe columns. "
  assert second_variable in data.columns, "Second input variable not found in dataframe columns. "

  # assign a colour to each variable
  colour_dict = {'deaths': '#EF553B' , 'cases': '#636EFA', 'tests': '#00CC96', 'mortality_rate': '#FFA15A', 'case_fatality_rate': '#FF97FF'}

  fig = make_subplots(rows = 1, cols =1)

  # select data for x and ys
  x = data.loc[country]['time']
  y1 = data.loc[country][first_variable]
  y2 = data.loc[country][second_variable]

  # specify colours for each variable
  y1_colour = colour_dict[first_variable]
  y2_colour = colour_dict[second_variable]

  fig.add_trace(go.Scatter(
    x=x,
    y=y1,
    name='Daily {first_variable}'.format(first_variable = first_variable.replace("_", " ")),
    line=dict(color=y1_colour, width=2) 

), row = 1, col=1)
  
  fig.add_trace(go.Scatter(
  x=x,
  y=y2,
  name='Daily {second_variable}'.format(second_variable = second_variable.replace("_", " ")),
  line=dict(color=y2_colour, width=2)
), row = 1, col=1)

  fig.update_layout(
      autosize=True,
      width=1500,
      height=800,
      template = "plotly_white",
      showlegend=True, 
      title = "COVID-19 daily {first_variable} and {second_variable} in {country}".format(first_variable = first_variable.replace("_", " "), second_variable = second_variable.replace("_", " "), country = country)
      )
  
  if 'rate' in first_variable and 'rate' in second_variable:
    fig.update_yaxes(title_text = "rate")
  elif 'rate' not in first_variable and 'rate' not in second_variable:
    fig.update_yaxes(title_text="Count", row=1, col=1)
  
  fig.update_xaxes(title_text="Time", row=1, col=1)

  fig.show()


# Data cleaning and Engineering

Next, import the data required for analysis. We originally explored data from a number of sources, but for the sake of making direct comparisons using the data we have elected to conduct all of our analysis using a single dataset that contains all the information we require. This data has been sourced from Our World in Data (https://ourworldindata.org/coronavirus).

In [ ]:
# Created by Laine, 2020-10-30
# import csv file straight from Our World in Data's github repo (updates automatically)
raw_data = pd.read_csv('https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/owid-covid-data.csv')

Now, we explore the shape of the data and the variables in the dataset.

In [ ]:
# Created by Julia, 2020-10-29
print("Shape of dataset: ", raw_data.shape)
raw_data.head()

Shape of dataset:  (55033, 49)


,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,total_cases_per_million,new_cases_per_million,new_cases_smoothed_per_million,total_deaths_per_million,new_deaths_per_million,new_deaths_smoothed_per_million,icu_patients,icu_patients_per_million,hosp_patients,hosp_patients_per_million,weekly_icu_admissions,weekly_icu_admissions_per_million,weekly_hosp_admissions,weekly_hosp_admissions_per_million,total_tests,new_tests,total_tests_per_thousand,new_tests_per_thousand,new_tests_smoothed,new_tests_smoothed_per_thousand,tests_per_case,positive_rate,tests_units,stringency_index,population,population_density,median_age,aged_65_older,aged_70_older,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index
0,ABW,North America,Aruba,2020-03-13,2.0,2.0,NaN,NaN,0.0,NaN,18.733,18.733,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,106766.0,584.8,41.2,13.085,7.452,35973.781,NaN,NaN,11.62,NaN,NaN,NaN,NaN,76.29,NaN
1,ABW,North America,Aruba,2020-03-19,NaN,NaN,0.286,NaN,NaN,0.0,NaN,NaN,2.676,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33.33,106766.0,584.8,41.2,13.085,7.452,35973.781,NaN,NaN,11.62,NaN,NaN,NaN,NaN,76.29,NaN
2,ABW,North America,Aruba,2020-03-20,4.0,2.0,0.286,NaN,0.0,0.0,37.465,18.733,2.676,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33.33,106766.0,584.8,41.2,13.085,7.452,35973.781,NaN,NaN,11.62,NaN,NaN,NaN,NaN,76.29,NaN
3,ABW,North America,Aruba,2020-03-21,NaN,NaN,0.286,NaN,NaN,0.0,NaN,NaN,2.676,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,44.44,106766.0,584.8,41.2,13.085,7.452,35973.781,NaN,NaN,11.62,NaN,NaN,NaN,NaN,76.29,NaN
4,ABW,North America,Aruba,2020-03-22,NaN,NaN,0.286,NaN,NaN,0.0,NaN,NaN,2.676,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,44.44,106766.0,584.8,41.2,13.085,7.452,35973.781,NaN,NaN,11.62,NaN,NaN,NaN,NaN,76.29,NaN


We can see that this dataset is quite large. It contains 53087 rows and 41 columns. We can also see a number of missing values, which we will leave as NaN. These missing values are one source of uncertainty in the data - we do not know if there is a systematic reason that they are missing, and we do not know whether having this data would affect our analysis. Keeping in mind that we do not have a complete dataset will remind us to be careful about drawing conclusions.

We will now filter the data down to the variables required for our analysis - determining how deadly COVID-19 is in 6 countries: Australia, Canada, India, Singapore, the United Kingdom, and the United States.

In [ ]:
# Created by Julia, 2020-10-29
# First, create a datetime variable

raw_data['time'] = pd.to_datetime(raw_data['date'], dayfirst = True)

# function to create separate datasets for each country

def select_country(raw_data, iso_code, select_cols):
  raw_data = raw_data
  iso_code = iso_code
  data = raw_data.loc[:, select_cols]
  data = data[data.loc[:, 'iso_code'] == iso_code ]

  return data

We then use the function to tidy the data, creating individual dataframes for each country. Each dataframe will contain corresponding variables.

In [ ]:
# Created by Laine, 2020-10-30

def select_country(raw_data, iso_code, select_cols):
  raw_data = raw_data
  iso_code = iso_code
  data = raw_data.loc[:, select_cols]
  data = data[data.loc[:, 'iso_code'] == iso_code ]

  return data

In [ ]:

# Created by Julia 2020-10-29
# Adapted by Laine 2020-10-30 -- added select_cols, converted country names to lower case for easier coding 
select_cols = ['time',
                'iso_code',  #left in so we can easily check the data
                'new_cases',
                'total_cases',
                'new_deaths',
                'total_deaths',
                'new_tests',
                'total_tests',
                'population',
                'median_age',
                'aged_65_older',
                'aged_70_older',
                'gdp_per_capita',
                'cardiovasc_death_rate',
                'diabetes_prevalence'
                ]

australia = select_country(raw_data, 'AUS', select_cols)
canada = select_country(raw_data, 'CAN', select_cols)
india = select_country(raw_data, 'IND', select_cols)
singapore = select_country(raw_data, 'SGP', select_cols)
united_kingdom = select_country(raw_data, 'GBR', select_cols)
united_states = select_country(raw_data, 'USA', select_cols)

We now have 6 individual tidy dataframes, one for each country of interest: Australia, Canada, India, Singapore, the United Kingdom, and the United States. Each dataframe now has 303 rows and 15 columns. The rows are observations, and the columns are: time, ISO code, new cases, total cases, new deaths, total deaths, new tests, total tests, population, median age, people aged 65+, people aged 70+, GDP per capita, cardiovascular death rate, and diabetes prevalence.

## Data transformations 


We created two variables called 'case_fatality_rate': (confirmed deaths/confirmed_cases)*100 and 'mortality_rate' (confirmed deaths / total population)*100 000

We created a function to calculate the case fatality rate, mortality rate per 100 000 people and tests per 100 000 people per day for each country, and then insert these 3 new variables into each dataframe.

These functions were then tested and combined into one function to streamline the process of creating and adding CFR, MR and test rate into each dataframe.

In [ ]:
# created by Julia, 2020-10-29
# edited by Laine, 2020-10-30: new vars weren't appearing in data after this function was applied, so i changed it a bit (your original code is commented out)
# function to calculate CFR, MR and test rate and insert into each dataframe

def insert_variables(data):
  """function to calculate case fatality rate (CFR), mortality rate (MR) and test rate and insert into each dataframe. """

  CFR = round((data['new_deaths']/data['new_cases'])*100,1) #1 significant figure

  MR = round((data['new_deaths']/data['population'])*100000,1) #1 significant figure

  test_rate = round((data['new_tests']/data['population'])*100000,1) #1 significant figure

  data['case_fatality_rate'] = CFR
  data['mortality_rate'] = MR
  data['test_rate'] = test_rate

  return data

We used this function to create the 3 new variables for each dataframe.

In [ ]:
# created on 2020-10-30 by Laine
# purpose: to reduce code duplication in the cell below
list_of_countries = [australia, canada, india, singapore, united_kingdom, united_states]

for c in list_of_countries:
  c = insert_variables(c)

all_countries = pd.concat(list_of_countries, keys = ['australia', 'canada', 'india', 'singapore', 'united_kingdom', 'united_states'])
all_countries.rename(columns = {'new_deaths': 'deaths', 'new_tests': 'tests', 'new_cases': 'cases'}, inplace=True)


# Country-specific analysis

Select the country below.


In [ ]:
country =  'United States' #@param  ['Australia', 'Canada', 'India', 'Singapore', 'United Kingdom', 'United States']

country = country.lower().replace(' ', '_')



# Single variable analysis

## Cases

Author: 

Purpose of the code:

Relation to the driving question:

In [ ]:
analyse_one_var(all_countries, country, "cases")

Author, start and end dates: 

Conclusion:

Relation to the driving question:

## Deaths

Author, start and end dates:

Purpose of the code:

Relation to the driving question:

In [ ]:
analyse_one_var(all_countries, country, "deaths")

Author, start and end dates: 

Conclusion:

Relation to the driving question:

## Mortality rate

Author, start and end dates:

Purpose of the code:

Relation to the driving question:

In [ ]:
analyse_one_var(all_countries, country, "mortality_rate")

Author, start and end dates: 

Conclusion:

Relation to the driving question:

## Tests

Author, start and end dates:

Purpose of the code:

Relation to the driving question:

In [ ]:
analyse_one_var(all_countries, country, "tests")

Author, start and end dates: 

Conclusion:

Relation to the driving question:

# Two variable analysis

## Cases and deaths over time


Author, start and end dates:

Purpose of the code:

Relation to the driving question:


In [ ]:
analyse_two_vars(all_countries, country, 'cases', 'deaths')

Author, start and end dates: 

Conclusion:

Relation to the driving question:

## Cases and tests over time

Author, start and end dates:

Purpose of the code:

Relation to the driving question:

In [ ]:
analyse_two_vars(all_countries, country, 'cases', 'tests')

Author, start and end dates: 

Conclusion:

Relation to the driving question:

## Deaths and tests over time


Author, start and end dates:

Purpose of the code:

Relation to the driving question:

In [ ]:
analyse_two_vars(all_countries, country, 'tests', 'deaths')

Author, start and end dates: 

Conclusion:

Relation to the driving question:

Document the independence of each variable